In [28]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib
import os
import pymorphy2

RusLem = pymorphy2.MorphAnalyzer()

def bprint(l, sep = " "):
    print sep.join(l)

In [23]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
 

#get list of sentences from html pages
def prepare_html_pages(html_dir):
    html_files = os.listdir(html_dir)
    sentences = []
    for filename in html_files:
        text = "" 
        with open(html_dir + filename, "r") as f:
            for line in f:
                text += line
                
        visible_text = text_from_html(text)
        sentences += visible_text.strip().split(".")
    
    return sentences

#sentences_normaliation
def normalization(sentences):
    for i, sent in enumerate(sentences):
        tokens = re.findall('[\w]+', sent.strip().lower(), re.U)
        sentences[i] = [RusLem.parse(token)[0].normal_form for token in tokens]
    return sentences

#creating forward index {sent_id: list of word} and token_dictionary
def create_forward_index(sentences):
    forward_index = {}
    token_list = set()
    for sent_id, sent in enumerate(sentences):
        forward_index[sent_id] = sent.strip().split()
        for token in sent.strip().split():
            token_list.add(token)
        
    return forward_index, list(token_list)

def calculate_idf(forward_index, token_list):
    cnt_sentences = len(forward_index)
    token_df = {token:0 for token in token_list}
    for tokens in forward_index.values():
        for token in token:
            token_df[token] += 1
    token_idf = {token:log(cnt_sentences / float(token_df[token])) for token in token_df}
    return token_idf

#create sentence vector
def doc2vec(sentence, token_list, token_idf, idf=False):
    vector = [sentence.count(token) for token in token_list]
    if idf:
        vector = [np.log(1 + c) * np.log(token_idf[token_list[i]]) for i, c in enumerate(vector)]
    return vector




        


In [24]:
x = prepare_html_pages("./htmls/")
print len(x), x[1], x[100]
t = create_forward_index(x)
print len(t), bprint(t[10])

1169   Marc Okrand [mɑrk ˈoʊkrænd]; 3 июля  1948 ) — американский  лингвист , известен как создатель языков для народов фантастических миров кинематографа США ( клингонский , вулканский и атлантский языки)  В конце века после падения основной их части, была реанимирована политическая активность христианских и исламских фундаменталистов, Римского понтифика и Далай-Ламы
1169 Считается, что последний носитель языка мутсун умер в 1930 году
None
